In [1]:
"""
Week 0: First Earth Observation Visualization
Create a multi-panel figure showing your AOI with different visualizations.
"""

import ee
import geemap
import json
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

# Load your AOI
with open('../data/external/aoi.geojson', 'r') as f:
    aoi_geojson = json.load(f)
aoi = geemap.geojson_to_ee(aoi_geojson)

# Load Sentinel-2 imagery
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Filter for your AOI and recent date
image = (s2
    .filterBounds(aoi)
    .filterDate('2023-06-01', '2023-09-01')  # Summer imagery
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .median())

# Create visualizations
Map = geemap.Map()
Map.centerObject(aoi, 9)

# True color
true_color = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}
Map.addLayer(image, true_color, 'True Color')

# False color (vegetation in red)
false_color = {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 3000, 'gamma': 1.4}
Map.addLayer(image, false_color, 'False Color (NIR-R-G)')

# NDVI
ndvi = image.normalizedDifference(['B8', 'B4'])
ndvi_vis = {'min': -0.2, 'max': 0.8, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndvi, ndvi_vis, 'NDVI')

# Add AOI outline
Map.addLayer(aoi, {'color': 'red'}, 'AOI Boundary')

Map

Map(center=[-41.12099664217866, -73.056936000002], controls=(WidgetControl(options=['position', 'transparent_b…